In [8]:
import pandas as pd
import csv as csv
from sklearn.ensemble import RandomForestClassifier

In [9]:
# Load training data
#import os
#os.getcwd()
train_df = pd.read_csv("NOS/train.csv", header=0)


In [10]:
# Convert "Sex" to be a dummy variable (female = 0, Male = 1)
train_df["Gender"] = train_df["Sex"].map({"female": 0, "male": 1}).astype(int)
train_df.head(3)

,PassengerID,Survived,Pclass,Name,Sex,Age,Sibsp,Parch,Ticket,Fare,Cabin,Embarked,Gender
0,1,0,3,"Yasbeck, Mr. Antoni",male,27.0,1,0,2659,14.4542,NaN,C,1
1,2,0,3,"Pasic, Mr. Jakob",male,21.0,0,0,315097,8.6625,NaN,S,1
2,3,1,1,"Dick, Mrs. Albert Adrian (Vera Gillespie)",female,17.0,1,0,17474,57.0000,B20,S,0


In [11]:
# Complement the missing values of "Age" column with average of "Age"
median_age = train_df["Age"].dropna().median()
if len(train_df.Age[train_df.Age.isnull()]) > 0:
  train_df.loc[(train_df.Age.isnull()), "Age"] = median_age


In [13]:
# remove un-used columns
train_df = train_df.drop(["Name", "Ticket", "Sex", "Sibsp", "Parch", "Fare", "Cabin", "Embarked", "PassengerID"], axis=1)
train_df.head(3)

,Survived,Pclass,Age,Gender
0,0,3,27.0,1
1,0,3,21.0,1
2,1,1,17.0,0


In [14]:
# Load test data, Convert "Sex" to be a dummy variable
test_df = pd.read_csv("NOS/test.csv", header=0)
test_df["Gender"] = test_df["Sex"].map({"female": 0, "male": 1}).astype(int)

In [15]:
# Complement the missing values of "Age" column with average of "Age"
median_age = test_df["Age"].dropna().median()
if len(test_df.Age[test_df.Age.isnull()]) > 0:
  test_df.loc[(test_df.Age.isnull()), "Age"] = median_age

In [18]:
# Copy test data's "PassengerId" column, and remove un-used columns
ids = test_df["PassengerID"].values
test_df = test_df.drop(["Name", "Ticket", "Sex", "Sibsp", "Parch", "Fare", "Cabin", "Embarked", "PassengerID"], axis=1)
test_df.head(3)

,Pclass,Age,Gender
0,3,31.0,0
1,3,26.0,1
2,1,60.0,0


In [22]:
# Predict with "Random Forest"
import numpy as np
train_data = train_df.values
test_data = test_df.values
model = RandomForestClassifier(n_estimators=100, random_state=3221)
output = model.fit(train_data[0::, 1::], train_data[0::, 0]).predict(test_data).astype(int)
#output = model.fit(train_data[0::, 1::], train_data[0::, 0]).predict_proba(test_data).astype(np.float32)

In [23]:
#print(output[:,0])

In [25]:
# export result to be "titanic_submit.csv"
submit_file = open("NOS/titanic_submit.csv", "w", newline='')
file_object = csv.writer(submit_file)
file_object.writerow(["PassengerId", "Survived"])
file_object.writerows(zip(ids, output))
submit_file.close()